In [1]:
import requests
import json
import pandas as pd
import psycopg2
from math import radians, cos, sin, asin, sqrt
from haversine import haversine
from pandas import json_normalize

In [2]:
# Requisição dos dados das Paradas - Indice i
dataset_full = []
for i in range(89):
    url        = 'https://jeap.rio.rj.gov.br/MOB/get_stops_details.rule?sys=MOB&INDICE=' + str(i)
    parametros = {}
    resposta   = requests.request("GET", url, params=parametros)
    objetos    = json.loads(resposta.text)
    dados      = json_normalize(objetos['data'])
    dataset_full.append(dados)

In [3]:
df = pd.concat(dataset_full,axis=0,ignore_index=True)

In [4]:
df.columns = ['stop_name','stop_id','stop_desc','stop_lat','stop_lon','location_type','modal','corredor','tipo_parada','tipo_sentido','propriedade_parada','seletivado','brs','sinalizacao','conservacao','tipo_abrigo','conservacao_abrigo','tipo_assento','tipo_baia','tipo_calcada','lixeira','braile','tipo_rampa','piso_tatil','elevador','numero_vagas','numero_cabines','ap','ra','bairro','observacoes','endereco','multimodal','id_sequencia','numerolinha','vista','horarios','existe_ponto','parent_station','nome_ponto','route_type']

In [5]:
df.head()

,stop_name,stop_id,stop_desc,stop_lat,stop_lon,location_type,modal,corredor,tipo_parada,tipo_sentido,...,endereco,multimodal,id_sequencia,numerolinha,vista,horarios,existe_ponto,parent_station,nome_ponto,route_type
0,Rua Sacadura Cabral 68,1001O00005C0,None,-22.89751,-43.18356,0,SPPO,Não se aplica,Ponto convencional,Desconhecido,...,Rua Sacadura Cabral 68,22,0005,NÃO,NÃO,NÃO,NÃO,None,None,3
1,"Avenida Barão de Tefé, oposto ao 99",1001O00006C0,None,-22.89731,-43.18737,0,SPPO,Não se aplica,Ponto convencional,Desconhecido,...,AVENIDA BARAO DE TEFE 107,22,0006,NÃO,NÃO,NÃO,SIM,None,"Avenida Barão de Tefé, oposto ao 99",3
2,"Avenida Venezuela , 153",1001O00008C0,None,-22.8959,-43.18638,0,SPPO,Não se aplica,Ponto convencional,Desconhecido,...,"Avenida Venezuela , 153",22,0008,NÃO,NÃO,NÃO,SIM,None,None,3
3,Barão de Tefé,1001O00010C1,None,-22.897487832035434,-43.1866272589302,0,SPPO,Não se aplica,Ponto convencional,Sul ou Oeste,...,R. Sacadura Cabral,22,0010,NÃO,NÃO,NÃO,SIM,None,Barão de Tefé,3
4,Parada dos Museus,1001VZ0011P3,None,-22.89595787504,-43.18199355919,0,VLT,Outros,Plataforma de Terminal,Ambos,...,Parada dos Museus,12,0011,None,None,None,SIM,1001VZ0012E9,Parada dos Museus,0


In [6]:
df_SPPO = df['modal']=='SPPO'
df_filtro = df[df_SPPO]

In [7]:
df_filtro.head()

,stop_name,stop_id,stop_desc,stop_lat,stop_lon,location_type,modal,corredor,tipo_parada,tipo_sentido,...,endereco,multimodal,id_sequencia,numerolinha,vista,horarios,existe_ponto,parent_station,nome_ponto,route_type
0,Rua Sacadura Cabral 68,1001O00005C0,None,-22.89751,-43.18356,0,SPPO,Não se aplica,Ponto convencional,Desconhecido,...,Rua Sacadura Cabral 68,22,0005,NÃO,NÃO,NÃO,NÃO,None,None,3
1,"Avenida Barão de Tefé, oposto ao 99",1001O00006C0,None,-22.89731,-43.18737,0,SPPO,Não se aplica,Ponto convencional,Desconhecido,...,AVENIDA BARAO DE TEFE 107,22,0006,NÃO,NÃO,NÃO,SIM,None,"Avenida Barão de Tefé, oposto ao 99",3
2,"Avenida Venezuela , 153",1001O00008C0,None,-22.8959,-43.18638,0,SPPO,Não se aplica,Ponto convencional,Desconhecido,...,"Avenida Venezuela , 153",22,0008,NÃO,NÃO,NÃO,SIM,None,None,3
3,Barão de Tefé,1001O00010C1,None,-22.897487832035434,-43.1866272589302,0,SPPO,Não se aplica,Ponto convencional,Sul ou Oeste,...,R. Sacadura Cabral,22,0010,NÃO,NÃO,NÃO,SIM,None,Barão de Tefé,3
11,"Rua Rivadávia Corrêa , 187",1002O00001C0,None,-22.89797,-43.1954,0,SPPO,Não se aplica,Ponto convencional,Desconhecido,...,Rua Rivadávia Corrêa próximo ao 111-139,22,0001,NÃO,NÃO,NÃO,SIM,None,"Rua Rivadávia Corrêa , 187",3


In [9]:
# Transformando a latitude e longitude em Numerico
df_filtro["stop_lat"] = pd.to_numeric(df_filtro["stop_lat"])
df_filtro["stop_lon"] = pd.to_numeric(df_filtro["stop_lon"])

C:\Users\vvanc\AppData\Local\Temp/ipykernel_34888/1343274456.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtro["stop_lat"] = pd.to_numeric(df_filtro["stop_lat"])
C:\Users\vvanc\AppData\Local\Temp/ipykernel_34888/1343274456.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtro["stop_lon"] = pd.to_numeric(df_filtro["stop_lon"])


In [10]:
df_aux = df_filtro

In [11]:
df_aux.insert(1, "DistanciaMinima",0, allow_duplicates=True)

In [13]:
aux = 0
for aux,row in df_aux.iterrows():
    ponto_aux = (df_aux.stop_lat[aux],df_aux.stop_lon[aux])
    dist_aux = 10000000000000
    for index,row in df_filtro.iterrows():
        ponto_onibus = (df_filtro.stop_lat[index],df_filtro.stop_lon[index])
        dist = haversine (ponto_aux,ponto_onibus)
        if dist < dist_aux:
            dist_aux = dist
            df_aux.loc['DistanciaMinima'] = dist
    aux = aux + 1

TypeError: can only concatenate str (not "int") to str